## Lab | Customer Analysis Final

In [1]:
# Hey guys, gotta be honest with you: I am lost in the "statistics woods"
# My statistics knowledge dates from 1990-1995 + Jan's explanation
# Knowledge is too few to start lab (round 7)
# It doesn't make sense to start improving the model when previous lab still has errors
# It's friday night. In weekend I will study more. Hope it will compensate
#
# Happy though that I made it this far  ;-)
# 
# Wish you all a nice weekend! Thanks 